# Chapter 20: Testing Strategies

Testing is essential for maintaining confidence in your application as it grows. Next.js 13+ with the App Router introduces new testing considerations—Server Components, Server Actions, and streaming architecture require different approaches than traditional React testing. A comprehensive testing strategy combines unit tests for utilities, component tests for UI behavior, integration tests for data flow, and end-to-end tests for critical user journeys.

By the end of this chapter, you'll master unit testing with Vitest, component testing with React Testing Library, end-to-end testing with Playwright, integration testing for Server Actions and API routes, testing Server Components, effective mocking strategies, and Test-Driven Development workflows tailored for Next.js.

## 20.1 Unit Testing with Jest/Vitest

Unit tests verify that individual functions and utilities work correctly in isolation.

### Vitest Setup

Vitest offers faster performance and native ESM support, making it ideal for modern Next.js projects:

```typescript
// vitest.config.ts
import { defineConfig } from 'vitest/config';
import react from '@vitejs/plugin-react';
import path from 'path';

export default defineConfig({
  plugins: [react()],
  test: {
    environment: 'jsdom',
    globals: true,
    setupFiles: ['./test/setup.ts'],
    alias: {
      '@': path.resolve(__dirname, './'),
    },
  },
});
```

```typescript
// test/setup.ts
import '@testing-library/jest-dom/vitest';
import { vi } from 'vitest';

// Mock Next.js router
vi.mock('next/navigation', () => ({
  useRouter: () => ({
    push: vi.fn(),
    replace: vi.fn(),
    refresh: vi.fn(),
    back: vi.fn(),
    forward: vi.fn(),
    prefetch: vi.fn(),
  }),
  usePathname: () => '/',
  useSearchParams: () => new URLSearchParams(),
}));

// Mock next/headers
vi.mock('next/headers', () => ({
  cookies: () => ({
    get: vi.fn(),
    set: vi.fn(),
    delete: vi.fn(),
  }),
  headers: () => new Headers(),
}));
```

### Testing Utilities

Test utility functions and helpers:

```typescript
// lib/utils.test.ts
import { describe, it, expect } from 'vitest';
import { formatDate, slugify, calculateTotal } from './utils';

describe('formatDate', () => {
  it('formats ISO date to readable string', () => {
    const date = '2024-01-15T10:30:00Z';
    expect(formatDate(date)).toBe('January 15, 2024');
  });

  it('handles invalid dates gracefully', () => {
    expect(formatDate('invalid')).toBe('Invalid date');
  });
});

describe('slugify', () => {
  it('converts text to URL-friendly slug', () => {
    expect(slugify('Hello World')).toBe('hello-world');
    expect(slugify('Special @#$ Characters')).toBe('special-characters');
  });

  it('handles edge cases', () => {
    expect(slugify('')).toBe('');
    expect(slugify('a')).toBe('a');
  });
});

describe('calculateTotal', () => {
  it('calculates total with tax', () => {
    const items = [
      { price: 100, quantity: 2 },
      { price: 50, quantity: 1 },
    ];
    const result = calculateTotal(items, 0.1); // 10% tax
    
    expect(result.subtotal).toBe(250);
    expect(result.tax).toBe(25);
    expect(result.total).toBe(275);
  });
});
```

### Testing Server Actions

Unit test Server Actions without the full Next.js runtime:

```typescript
// app/actions.test.ts
import { describe, it, expect, vi, beforeEach } from 'vitest';
import { createPost, deletePost } from './actions';
import { prisma } from '@/lib/db';

// Mock Prisma
vi.mock('@/lib/db', () => ({
  prisma: {
    post: {
      create: vi.fn(),
      delete: vi.fn(),
      findUnique: vi.fn(),
    },
  },
}));

// Mock auth
vi.mock('@/lib/auth', () => ({
  getServerSession: vi.fn(() => ({
    user: { id: 'user-1', role: 'USER' },
  })),
}));

// Mock revalidatePath
vi.mock('next/cache', () => ({
  revalidatePath: vi.fn(),
}));

describe('createPost', () => {
  beforeEach(() => {
    vi.clearAllMocks();
  });

  it('creates a post with valid data', async () => {
    const formData = new FormData();
    formData.set('title', 'Test Post');
    formData.set('content', 'Test content');

    const mockPost = { id: 'post-1', title: 'Test Post' };
    vi.mocked(prisma.post.create).mockResolvedValue(mockPost);

    const result = await createPost(formData);

    expect(prisma.post.create).toHaveBeenCalledWith({
      data: expect.objectContaining({
        title: 'Test Post',
        content: 'Test content',
        authorId: 'user-1',
      }),
    });
    expect(result).toEqual({ success: true, post: mockPost });
  });

  it('throws error if user is not authenticated', async () => {
    vi.mocked(getServerSession).mockResolvedValue(null);

    const formData = new FormData();
    formData.set('title', 'Test');

    await expect(createPost(formData)).rejects.toThrow('Unauthorized');
  });
});
```

## 20.2 Component Testing

Test React components in isolation using React Testing Library.

### Testing Client Components

Test interactive components with user events:

```typescript
// components/counter.test.tsx
import { describe, it, expect, vi } from 'vitest';
import { render, screen, fireEvent } from '@testing-library/react';
import { Counter } from './counter';

describe('Counter', () => {
  it('renders initial count', () => {
    render(<Counter initialCount={5} />);
    expect(screen.getByText('5')).toBeInTheDocument();
  });

  it('increments count when clicking +', () => {
    render(<Counter initialCount={0} />);
    
    fireEvent.click(screen.getByText('+'));
    
    expect(screen.getByText('1')).toBeInTheDocument();
  });

  it('decrements count when clicking -', () => {
    render(<Counter initialCount={5} />);
    
    fireEvent.click(screen.getByText('-'));
    
    expect(screen.getByText('4')).toBeInTheDocument();
  });

  it('calls onChange callback when count changes', () => {
    const handleChange = vi.fn();
    render(<Counter initialCount={0} onChange={handleChange} />);
    
    fireEvent.click(screen.getByText('+'));
    
    expect(handleChange).toHaveBeenCalledWith(1);
  });
});
```

### Testing Async Components

Test components with async operations:

```typescript
// components/user-profile.test.tsx
import { describe, it, expect, vi } from 'vitest';
import { render, screen, waitFor } from '@testing-library/react';
import { UserProfile } from './user-profile';

// Mock fetch
global.fetch = vi.fn();

describe('UserProfile', () => {
  it('displays loading state initially', () => {
    vi.mocked(fetch).mockImplementation(() => new Promise(() => {}));
    
    render(<UserProfile userId="123" />);
    
    expect(screen.getByText('Loading...')).toBeInTheDocument();
  });

  it('displays user data after loading', async () => {
    const mockUser = { name: 'John Doe', email: 'john@example.com' };
    
    vi.mocked(fetch).mockResolvedValue({
      ok: true,
      json: async () => mockUser,
    } as Response);

    render(<UserProfile userId="123" />);
    
    await waitFor(() => {
      expect(screen.getByText('John Doe')).toBeInTheDocument();
    });
    
    expect(screen.getByText('john@example.com')).toBeInTheDocument();
  });

  it('displays error message on fetch failure', async () => {
    vi.mocked(fetch).mockRejectedValue(new Error('Network error'));

    render(<UserProfile userId="123" />);
    
    await waitFor(() => {
      expect(screen.getByText('Failed to load user')).toBeInTheDocument();
    });
  });
});
```

### Testing Forms

Test form validation and submission:

```typescript
// components/contact-form.test.tsx
import { describe, it, expect, vi } from 'vitest';
import { render, screen, fireEvent, waitFor } from '@testing-library/react';
import userEvent from '@testing-library/user-event';
import { ContactForm } from './contact-form';

describe('ContactForm', () => {
  it('validates required fields', async () => {
    render(<ContactForm onSubmit={vi.fn()} />);
    
    fireEvent.click(screen.getByText('Send Message'));
    
    await waitFor(() => {
      expect(screen.getByText('Name is required')).toBeInTheDocument();
      expect(screen.getByText('Email is required')).toBeInTheDocument();
    });
  });

  it('validates email format', async () => {
    render(<ContactForm onSubmit={vi.fn()} />);
    
    await userEvent.type(screen.getByLabelText('Email'), 'invalid-email');
    fireEvent.click(screen.getByText('Send Message'));
    
    await waitFor(() => {
      expect(screen.getByText('Invalid email address')).toBeInTheDocument();
    });
  });

  it('submits form with valid data', async () => {
    const handleSubmit = vi.fn();
    render(<ContactForm onSubmit={handleSubmit} />);
    
    await userEvent.type(screen.getByLabelText('Name'), 'John Doe');
    await userEvent.type(screen.getByLabelText('Email'), 'john@example.com');
    await userEvent.type(screen.getByLabelText('Message'), 'Hello!');
    
    fireEvent.click(screen.getByText('Send Message'));
    
    await waitFor(() => {
      expect(handleSubmit).toHaveBeenCalledWith({
        name: 'John Doe',
        email: 'john@example.com',
        message: 'Hello!',
      });
    });
  });
});
```

## 20.3 End-to-End Testing with Playwright

E2E tests verify complete user flows across the entire application stack.

### Playwright Configuration

Set up Playwright for Next.js:

```typescript
// playwright.config.ts
import { defineConfig, devices } from '@playwright/test';

export default defineConfig({
  testDir: './e2e',
  fullyParallel: true,
  forbidOnly: !!process.env.CI,
  retries: process.env.CI ? 2 : 0,
  workers: process.env.CI ? 1 : undefined,
  reporter: 'html',
  use: {
    baseURL: 'http://localhost:3000',
    trace: 'on-first-retry',
  },
  projects: [
    {
      name: 'chromium',
      use: { ...devices['Desktop Chrome'] },
    },
    {
      name: 'firefox',
      use: { ...devices['Desktop Firefox'] },
    },
  ],
  webServer: {
    command: 'npm run dev',
    url: 'http://localhost:3000',
    reuseExistingServer: !process.env.CI,
  },
});
```

### Testing Critical User Flows

Test authentication and core functionality:

```typescript
// e2e/auth.spec.ts
import { test, expect } from '@playwright/test';

test.describe('Authentication', () => {
  test('user can sign in', async ({ page }) => {
    await page.goto('/login');
    
    await page.fill('[name="email"]', 'test@example.com');
    await page.fill('[name="password"]', 'password123');
    await page.click('button[type="submit"]');
    
    // Wait for navigation
    await page.waitForURL('/dashboard');
    
    // Verify logged in state
    await expect(page.locator('text=Welcome, Test User')).toBeVisible();
  });

  test('protected routes redirect to login', async ({ page }) => {
    await page.goto('/dashboard');
    
    // Should redirect to login
    await page.waitForURL('/login?callbackUrl=%2Fdashboard');
    
    await expect(page.locator('h1')).toContainText('Sign In');
  });
});

test.describe('Blog Flow', () => {
  test('user can create and view a post', async ({ page }) => {
    // Login first
    await page.goto('/login');
    await page.fill('[name="email"]', 'author@example.com');
    await page.fill('[name="password"]', 'password123');
    await page.click('button[type="submit"]');
    
    // Create post
    await page.goto('/blog/new');
    await page.fill('[name="title"]', 'My Test Post');
    await page.fill('[name="content"]', 'This is the content');
    await page.click('button[type="submit"]');
    
    // Verify post was created
    await expect(page.locator('h1')).toContainText('My Test Post');
    
    // Verify it appears in list
    await page.goto('/blog');
    await expect(page.locator('text=My Test Post')).toBeVisible();
  });
});
```

### Testing Responsive Behavior

Test across different viewports:

```typescript
// e2e/responsive.spec.ts
import { test, expect } from '@playwright/test';

test.describe('Responsive Design', () => {
  test('mobile menu works on small screens', async ({ page }) => {
    await page.setViewportSize({ width: 375, height: 667 });
    await page.goto('/');
    
    // Mobile menu should be hidden initially
    await expect(page.locator('nav[aria-label="Main"]')).not.toBeVisible();
    
    // Click hamburger menu
    await page.click('button[aria-label="Open menu"]');
    
    // Menu should be visible
    await expect(page.locator('nav[aria-label="Main"]')).toBeVisible();
  });

  test('desktop shows full navigation', async ({ page }) => {
    await page.setViewportSize({ width: 1280, height: 720 });
    await page.goto('/');
    
    // Navigation should be visible without clicking
    await expect(page.locator('nav[aria-label="Main"]')).toBeVisible();
    await expect(page.locator('text=Dashboard')).toBeVisible();
  });
});
```

### API Testing

Test API routes directly:

```typescript
// e2e/api.spec.ts
import { test, expect } from '@playwright/test';

test.describe('API Routes', () => {
  test('GET /api/posts returns posts list', async ({ request }) => {
    const response = await request.get('/api/posts');
    
    expect(response.ok()).toBeTruthy();
    
    const data = await response.json();
    expect(Array.isArray(data.posts)).toBeTruthy();
    expect(data.posts.length).toBeGreaterThan(0);
  });

  test('POST /api/posts creates new post', async ({ request }) => {
    const response = await request.post('/api/posts', {
      data: {
        title: 'API Test Post',
        content: 'Content from API test',
      },
    });
    
    expect(response.status()).toBe(201);
    
    const data = await response.json();
    expect(data.post.title).toBe('API Test Post');
  });

  test('POST /api/posts validates input', async ({ request }) => {
    const response = await request.post('/api/posts', {
      data: { title: '' }, // Invalid: empty title
    });
    
    expect(response.status()).toBe(400);
  });
});
```

## 20.4 Integration Testing

Test how components and modules work together.

### Testing Data Flow

Test complete data fetching and rendering cycles:

```typescript
// test/integration/post-creation.test.tsx
import { describe, it, expect, vi, beforeAll, afterAll } from 'vitest';
import { render, screen, waitFor, fireEvent } from '@testing-library/react';
import { CreatePostPage } from '@/app/blog/new/page';
import { prisma } from '@/lib/db';

describe('Post Creation Integration', () => {
  beforeAll(async () => {
    // Setup test database
    await prisma.$connect();
  });

  afterAll(async () => {
    await prisma.$disconnect();
  });

  it('creates post and updates list', async () => {
    // Render the page
    render(<CreatePostPage />);
    
    // Fill form
    fireEvent.change(screen.getByLabelText('Title'), {
      target: { value: 'Integration Test Post' },
    });
    fireEvent.change(screen.getByLabelText('Content'), {
      target: { value: 'Test content' },
    });
    
    // Submit
    fireEvent.click(screen.getByText('Create Post'));
    
    // Wait for success message
    await waitFor(() => {
      expect(screen.getByText('Post created successfully')).toBeInTheDocument();
    });
    
    // Verify in database
    const post = await prisma.post.findFirst({
      where: { title: 'Integration Test Post' },
    });
    
    expect(post).toBeTruthy();
    expect(post?.content).toBe('Test content');
  });
});
```

### Testing Route Handlers

Test API route handlers in isolation:

```typescript
// app/api/users/route.test.ts
import { describe, it, expect, vi, beforeEach } from 'vitest';
import { GET, POST } from './route';
import { NextRequest } from 'next/server';

// Mock database
vi.mock('@/lib/db', () => ({
  prisma: {
    user: {
      findMany: vi.fn(),
      create: vi.fn(),
    },
  },
}));

describe('Users API', () => {
  beforeEach(() => {
    vi.clearAllMocks();
  });

  describe('GET', () => {
    it('returns list of users', async () => {
      const mockUsers = [
        { id: '1', name: 'User 1', email: 'user1@example.com' },
        { id: '2', name: 'User 2', email: 'user2@example.com' },
      ];
      
      vi.mocked(prisma.user.findMany).mockResolvedValue(mockUsers);

      const response = await GET();
      const data = await response.json();

      expect(response.status).toBe(200);
      expect(data.users).toEqual(mockUsers);
    });

    it('handles database errors', async () => {
      vi.mocked(prisma.user.findMany).mockRejectedValue(new Error('DB Error'));

      const response = await GET();
      
      expect(response.status).toBe(500);
    });
  });

  describe('POST', () => {
    it('creates new user with valid data', async () => {
      const mockUser = { id: '1', name: 'New User', email: 'new@example.com' };
      vi.mocked(prisma.user.create).mockResolvedValue(mockUser);

      const request = new NextRequest('http://localhost:3000/api/users', {
        method: 'POST',
        body: JSON.stringify({
          name: 'New User',
          email: 'new@example.com',
        }),
      });

      const response = await POST(request);
      const data = await response.json();

      expect(response.status).toBe(201);
      expect(data.user).toEqual(mockUser);
    });

    it('returns 400 for invalid data', async () => {
      const request = new NextRequest('http://localhost:3000/api/users', {
        method: 'POST',
        body: JSON.stringify({ name: '', email: 'invalid' }),
      });

      const response = await POST(request);
      
      expect(response.status).toBe(400);
    });
  });
});
```

## 20.5 Testing Server Components

Server Components require special handling since they run on the server during render.

### Testing Server Component Output

Test the rendered output of Server Components:

```typescript
// app/blog/page.test.tsx
import { describe, it, expect, vi } from 'vitest';
import { render, screen } from '@testing-library/react';
import BlogPage from './page';

// Mock database
vi.mock('@/lib/db', () => ({
  prisma: {
    post: {
      findMany: vi.fn(),
    },
  },
}));

describe('BlogPage (Server Component)', () => {
  it('renders list of posts', async () => {
    const mockPosts = [
      { id: '1', title: 'Post 1', slug: 'post-1', published: true },
      { id: '2', title: 'Post 2', slug: 'post-2', published: true },
    ];
    
    vi.mocked(prisma.post.findMany).mockResolvedValue(mockPosts);

    // Server Components are async, so we await the render
    const Page = await BlogPage({ searchParams: {} });
    render(Page);

    expect(screen.getByText('Post 1')).toBeInTheDocument();
    expect(screen.getByText('Post 2')).toBeInTheDocument();
  });

  it('renders empty state when no posts', async () => {
    vi.mocked(prisma.post.findMany).mockResolvedValue([]);

    const Page = await BlogPage({ searchParams: {} });
    render(Page);

    expect(screen.getByText('No posts found')).toBeInTheDocument();
  });
});
```

### Snapshot Testing for Server Components

Use snapshots to verify Server Component output:

```typescript
// components/server/article.test.tsx
import { describe, it, expect, vi } from 'vitest';
import { render } from '@testing-library/react';
import { Article } from './article';

vi.mock('@/lib/data', () => ({
  getArticle: vi.fn(),
}));

describe('Article', () => {
  it('matches snapshot', async () => {
    vi.mocked(getArticle).mockResolvedValue({
      title: 'Test Article',
      content: '<p>Test content</p>',
      publishedAt: new Date('2024-01-01'),
    });

    const Component = await Article({ slug: 'test-article' });
    const { container } = render(Component);

    expect(container).toMatchSnapshot();
  });
});
```

## 20.6 Mocking and Stubbing

Effective mocking isolates tests from external dependencies.

### Mocking Next.js Modules

Create reusable mocks for Next.js APIs:

```typescript
// test/mocks/next-navigation.ts
import { vi } from 'vitest';

export const createMockRouter = (overrides = {}) => ({
  push: vi.fn(),
  replace: vi.fn(),
  refresh: vi.fn(),
  back: vi.fn(),
  forward: vi.fn(),
  prefetch: vi.fn(),
  ...overrides,
});

// Usage in test
vi.mock('next/navigation', () => ({
  useRouter: () => createMockRouter(),
  usePathname: () => '/dashboard',
  useSearchParams: () => new URLSearchParams('?tab=settings'),
}));
```

### Mocking External APIs

Mock fetch calls for external services:

```typescript
// lib/api.test.ts
import { describe, it, expect, vi, beforeEach, afterEach } from 'vitest';
import { fetchWeather } from './api';

describe('fetchWeather', () => {
  beforeEach(() => {
    global.fetch = vi.fn();
  });

  afterEach(() => {
    vi.restoreAllMocks();
  });

  it('returns weather data on success', async () => {
    const mockResponse = {
      temperature: 22,
      condition: 'Sunny',
    };

    vi.mocked(fetch).mockResolvedValue({
      ok: true,
      json: async () => mockResponse,
    } as Response);

    const result = await fetchWeather('London');
    
    expect(result).toEqual(mockResponse);
    expect(fetch).toHaveBeenCalledWith(
      expect.stringContaining('api.weather.com'),
      expect.any(Object)
    );
  });

  it('throws error on failed request', async () => {
    vi.mocked(fetch).mockResolvedValue({
      ok: false,
      status: 404,
    } as Response);

    await expect(fetchWeather('InvalidCity')).rejects.toThrow();
  });
});
```

### Database Mocking Strategies

Use in-memory databases or mocks for integration tests:

```typescript
// test/setup-db.ts
import { PrismaClient } from '@prisma/client';
import { execSync } from 'child_process';

const prisma = new PrismaClient();

export async function setupTestDb() {
  // Use test database URL
  process.env.DATABASE_URL = process.env.TEST_DATABASE_URL;
  
  // Reset database
  execSync('npx prisma migrate reset --force --skip-seed');
  
  return prisma;
}

export async function teardownTestDb() {
  await prisma.$disconnect();
}

// Usage in test file
import { setupTestDb, teardownTestDb } from '@/test/setup-db';

beforeAll(async () => {
  await setupTestDb();
});

afterAll(async () => {
  await teardownTestDb();
});
```

## 20.7 Test-Driven Development

TDD workflow for Next.js features: Red-Green-Refactor.

### TDD Example: Shopping Cart

Build a cart feature using TDD:

```typescript
// Step 1: Write failing test (Red)
// stores/cart.test.ts
import { describe, it, expect, beforeEach } from 'vitest';
import { useCartStore } from './cart';

describe('Cart Store', () => {
  beforeEach(() => {
    // Reset store state
    useCartStore.setState({ items: [] });
  });

  it('adds item to cart', () => {
    const { addItem } = useCartStore.getState();
    
    addItem({ id: '1', name: 'Product', price: 10, quantity: 1 });
    
    const { items } = useCartStore.getState();
    expect(items).toHaveLength(1);
    expect(items[0].name).toBe('Product');
  });

  it('calculates total price', () => {
    const { addItem, total } = useCartStore.getState();
    
    addItem({ id: '1', name: 'Product 1', price: 10, quantity: 2 });
    addItem({ id: '2', name: 'Product 2', price: 5, quantity: 1 });
    
    expect(useCartStore.getState().total).toBe(25);
  });
});

// Step 2: Implement to pass (Green)
// stores/cart.ts
import { create } from 'zustand';

interface CartItem {
  id: string;
  name: string;
  price: number;
  quantity: number;
}

interface CartState {
  items: CartItem[];
  addItem: (item: CartItem) => void;
  total: number;
}

export const useCartStore = create<CartState>((set, get) => ({
  items: [],
  addItem: (item) => set((state) => ({
    items: [...state.items, item],
  })),
  get total() {
    return get().items.reduce((sum, item) => sum + item.price * item.quantity, 0);
  },
}));

// Step 3: Refactor and add edge cases
it('increases quantity if item already exists', () => {
  const { addItem } = useCartStore.getState();
  
  addItem({ id: '1', name: 'Product', price: 10, quantity: 1 });
  addItem({ id: '1', name: 'Product', price: 10, quantity: 1 });
  
  const { items } = useCartStore.getState();
  expect(items).toHaveLength(1);
  expect(items[0].quantity).toBe(2);
});
```

### TDD for API Routes

Develop API endpoints test-first:

```typescript
// Step 1: Write test
// app/api/cart/route.test.ts
describe('Cart API', () => {
  describe('POST /api/cart', () => {
    it('adds item to user cart', async () => {
      const session = await createTestSession();
      
      const response = await fetch('/api/cart', {
        method: 'POST',
        headers: { 
          'Content-Type': 'application/json',
          'Cookie': `session=${session.token}`,
        },
        body: JSON.stringify({
          productId: 'prod-1',
          quantity: 2,
        }),
      });

      expect(response.status).toBe(201);
      
      const cart = await getUserCart(session.userId);
      expect(cart.items).toHaveLength(1);
      expect(cart.items[0].quantity).toBe(2);
    });
  });
});

// Step 2: Implement
// app/api/cart/route.ts
export async function POST(request: Request) {
  const session = await getServerSession();
  if (!session) {
    return NextResponse.json({ error: 'Unauthorized' }, { status: 401 });
  }

  const body = await request.json();
  
  await prisma.cartItem.upsert({
    where: {
      userId_productId: {
        userId: session.user.id,
        productId: body.productId,
      },
    },
    update: {
      quantity: { increment: body.quantity },
    },
    create: {
      userId: session.user.id,
      productId: body.productId,
      quantity: body.quantity,
    },
  });

  return NextResponse.json({ success: true }, { status: 201 });
}
```

## Key Takeaways from Chapter 20

1. **Unit Testing Setup**: Use Vitest for faster ESM-compatible testing with `globals: true` for Jest-like syntax. Create a `test/setup.ts` file to mock Next.js modules (`next/navigation`, `next/headers`, `next/cache`) before running tests, ensuring components using these APIs don't crash during unit tests.

2. **Component Testing**: Test Client Components using React Testing Library with `fireEvent` or `userEvent` for interactions. For Server Components, await the async component function before rendering in tests, then assert on the resulting JSX structure or use snapshot testing for complex markup.

3. **End-to-End Testing**: Configure Playwright with a `webServer` command to automatically start the Next.js dev server during tests. Test critical user flows (auth, checkout, content creation) across multiple browsers and viewport sizes, and use `request` fixture to test API routes directly without UI interaction.

4. **Integration Testing**: Test complete data flows from UI through Server Actions to database using a test database (SQLite or isolated PostgreSQL schema). Verify that form submissions actually persist data and that route handlers return correct HTTP status codes and response bodies.

5. **Server Component Testing**: Treat Server Components as async functions that return JSX. Mock the database or external APIs they call, await the component resolution, then use Testing Library to render the result and assert on the output. Use snapshot testing sparingly for complex Server Component trees.

6. **Mocking Strategies**: Create centralized mocks for Next.js router, cookies, and headers. Use `vi.mock` at the top of test files to auto-mock modules, and use `vi.mocked` to type-safe the mocked functions. For database integration tests, use `beforeAll` to reset the test database state and `afterAll` to clean up connections.

7. **Test-Driven Development**: Follow the Red-Green-Refactor cycle: write a failing test that defines expected behavior, implement the minimal code to make it pass, then refactor while keeping tests green. Start with unit tests for utilities and stores, then integration tests for API routes, and finally E2E tests for complete user flows.

## Coming Up Next

**Chapter 21: Performance Optimization**

Now that your application is well-tested and reliable, it's time to make it fast. In Chapter 21, we'll explore Core Web Vitals optimization, bundle size reduction techniques, image and asset optimization, code splitting strategies, memoization with React Compiler, and performance profiling tools. You'll learn how to deliver exceptional user experiences with lightning-fast page loads and smooth interactions.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. error_handling_and_debugging.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. performance_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
